### Load libraries

In [ ]:
from sys import argv
import numpy as np
import pandas as pd
import scipy as sp
from scipy import ndimage
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize
import shapefile as sf
from scipy.interpolate import RegularGridInterpolator
from gnam.model.gridmod3d import gridmod3d as gm
from gnam.model.bbox import bbox as bb
from shapely.geometry import Point, Polygon

In [ ]:
!ls ../../../data_notebooks/pickled

### Read in previously computed smoothed model

In [ ]:
#this is a pickled dictionary with 4D ndarray, and 1D meta data arrays
dpath = '../../../data_notebooks/pickled/'
fname = 'smooth_z10m_sig250m_intnam_model_vp_vs_rho_Q_props.npz'
ifilename = dpath + fname

#Unpickle
data = np.load(ifilename)
props = data['props'] #4D ndarray

#meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('xd:\n',xdata)
print('yd:\n',ydata)
print('zd:\n',zdata)

# Setup Coordinate related vars
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = (-zmin) + (nz-1)*dz

nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
gm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin))
print('gm3d:\n',gm3d)

#free up some memory
del props 

In [ ]:
print('gm3d:\n',gm3d)

### Plot a depth slice of the smoothed model

In [ ]:
hires_surf = gm3d[0,:,:,200].T
print('hires_surf.shape:',hires_surf.shape)
gxyc = gm3d.getGlobalCoordsPointsXY()
#get min max to normalize surface
hires_vp_min = np.min(hires_surf)
hires_vp_max = np.max(hires_surf)
print('vp_min:',hires_vp_min)
print('vp_max:',hires_vp_max)
hires_norm = Normalize(hires_vp_min,hires_vp_max)

fig, ax = plt.subplots(1,figsize=(8,6))
sc = ax.scatter(gxyc[:,0],gxyc[:,1],s=1,c=hires_surf.flatten(),cmap=plt.cm.jet,norm=hires_norm)
ax.set_title('Full NAM Model Surface (z=0)')
fig.colorbar(sc)
plt.show()

### Subsample model

In [ ]:
dx,dy,dz = gm3d.get_deltas()
print('dx,dy,dz = %f,%f,%f' %(dx,dy,dz))

gm3d.subsample(isz=10,idz=20,idx=4,idy=4) # idx=idy=2 by default
#gm3d.subsample(idx=8,idy=8,isz=20,idz=40)
print(gm3d)

### Compress and Pickle Subsampled model

In [ ]:
import numpy as np
sub_props = gm3d.getNPArray()
#ossfqn = './subsamp_smooth_z' + str(int(gm3d.get_deltas()[2])) 
#ossfqn = ossfqn + 'm_zcels_nam_model_vp_vs_rho_Q_props.npz'
dpath = '../../../data_notebooks/pickled/'
fname = 'subsamp_smooth_z' + str(int(gm3d.get_deltas()[2])) + 'm_zcels_nam_model_vp_vs_rho_Q_props.npz'
ossfqn = dpath + fname
print(ossfqn)

print('xdata:',xdata)
print('ydata:',ydata)
print('zdata:',zdata)

xdata[0] = gm3d.get_gorigin()[0]
ydata[0] = gm3d.get_gorigin()[1]
zdata[0] = gm3d.get_gorigin()[2]
xdata[1] = gm3d.get_deltas()[0]
ydata[1] = gm3d.get_deltas()[1]
zdata[1] = gm3d.get_deltas()[2]
xdata[2] = gm3d.get_npoints()[0]
ydata[2] = gm3d.get_npoints()[1]
zdata[2] = gm3d.get_npoints()[2]
print('xdata:',xdata)
print('ydata:',ydata)
print('zdata:',zdata)
np.savez_compressed(ossfqn,props=sub_props,xd=xdata,yd=ydata,zd=zdata)
del sub_props

### Unpickle Subsampled model

In [ ]:
#this is a pickled dictionary with 4D ndarray, and 1D meta data arrays
#ifilename = './subsamp_smooth_z100m_zcels_nam_model_vp_vs_rho_Q_props.npz'
#ifilename = './subsamp_smooth_z100m_nam_model_vp_vs_rho_Q_props.npz'
dpath = '../../../data_notebooks/pickled/'
fname = 'subsamp_smooth_z' + str(int(gm3d.get_deltas()[2])) + 'm_zcels_nam_model_vp_vs_rho_Q_props.npz'
ifilename = dpath + fname

#Unpickle
data = np.load(ifilename)
props = data['props'] #4D ndarray

#meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('xd:\n',xdata)
print('yd:\n',ydata)
print('zd:\n',zdata)

# Setup Coordinate related vars
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = zmin + (nz-1)*dz

nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
gm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin))
print('gm3d:\n',gm3d)

#free up some memory
del props 

In [ ]:
!ls ../../../data_notebooks/pickled/

### Plot a depth slice

In [ ]:
lores_surf = gm3d[0,:,:,10].T
#lores_surf = gm3d[0,:,:,5].T
print('lores_surf.shape:',lores_surf.shape)
gxyc = gm3d.getGlobalCoordsPointsXY()
#get min max to normalize surface
lores_vp_min = np.min(lores_surf)
lores_vp_max = np.max(lores_surf)
print('vp_min:',lores_vp_min)
print('vp_max:',lores_vp_max)
lores_norm = Normalize(lores_vp_min,lores_vp_max)

fig, ax = plt.subplots(1,figsize=(8,6))
sc = ax.scatter(gxyc[:,0],gxyc[:,1],s=1,c=lores_surf.flatten(),cmap=plt.cm.jet,norm=lores_norm)
ax.set_title('Full NAM Model Surface (z=0)')
fig.colorbar(sc)
plt.show()

### Unpickle bounding box

In [ ]:
import pickle

dpath = '../../../data_notebooks/pickled/'
fname = 'model_bbox_200m.pickle'
ifile = dpath + fname
print('unpickle:',ifile)

f = open(ifile, 'rb')
dill_bbox = pickle.load(f)
f.close()
print('bbox after pickle :\n',dill_bbox)

### Plot bounding box on top of surface

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots(1,figsize=(8,6))
sc = ax.scatter(gxyc[:,0],gxyc[:,1],s=1,c=lores_surf.flatten(),cmap=plt.cm.jet,norm=lores_norm,zorder=0)
ax.plot(dill_bbox.getCLoop()[:,0],dill_bbox.getCLoop()[:,1],c='black',linestyle='dotted',zorder=4)
ax.set_title('Full NAM Model Surface (z=0)')
fig.colorbar(sc)
plt.show()

### Slice a subsurface volume by a bounding box

In [ ]:
import time

start_t = time.time()
slgm3d = gm3d.slice_volume_by_bbox(dill_bbox)
exec_time = time.time() - start_t
print('execution time:', exec_time)
print('slgm3d:\n',slgm3d)

### Compress and pickle sliced model

In [ ]:
import numpy as np
slice_props = slgm3d.getNPArray()
print(ossfqn)
dpath = '../../../data_notebooks/pickled/'
fname = 'rect_rot_z' + str(int(gm3d.get_deltas()[2])) + 'm_zcels_nam_model_vp_vs_rho_Q_props.npz'
ossfqn = dpath + fname
print(ossfqn)

xdata[0] = slgm3d.get_gorigin()[0]
ydata[0] = slgm3d.get_gorigin()[1]
zdata[0] = slgm3d.get_gorigin()[2]
xdata[1] = slgm3d.get_deltas()[0]
ydata[1] = slgm3d.get_deltas()[1]
zdata[1] = slgm3d.get_deltas()[2]
xdata[2] = slgm3d.get_npoints()[0]
ydata[2] = slgm3d.get_npoints()[1]
zdata[2] = slgm3d.get_npoints()[2]
rotdeg   = dill_bbox.getRotDeg()
print('xdata:',xdata)
print('ydata:',ydata)
print('zdata:',zdata)
np.savez_compressed(ossfqn,props=slice_props,xd=xdata,yd=ydata,zd=zdata,rot=rotdeg)
del slice_props

### Unpickle Sliced model

In [ ]:
#this is a pickled dictionary with 4D ndarray, and 1D meta data arrays
#ifilename = './rect_rot_z100m_zcels_nam_model_vp_vs_rho_Q_props.npz'
dpath = '../../../data_notebooks/pickled/'
fname = 'rect_rot_z' + str(int(gm3d.get_deltas()[2])) + 'm_zcels_nam_model_vp_vs_rho_Q_props.npz'
ifilename = dpath + fname

#Unpickle
data = np.load(ifilename)
props = data['props'] #4D ndarray
print('props.shape:',props.shape)

#meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']
rdeg  = data['rot']

print('xd:\n',xdata)
print('yd:\n',ydata)
print('zd:\n',zdata)

# Setup Coordinate related vars
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = zmin + (nz-1)*dz

nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
slgm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin),rdeg)
print('slgm3d:\n',slgm3d)

#free up some memory
del props 

### Plot and compare depth slice of sliced model

In [ ]:
slice_surf = slgm3d[0,:,:,0].T

slxy = slgm3d.getGlobalCoordsPointsXY()

In [ ]:
%matplotlib notebook

slice_vp_min = np.min(slice_surf)
slice_vp_max = np.max(slice_surf)
print('vp_min:',slice_vp_min)
print('vp_max:',slice_vp_max)
slice_norm = Normalize(slice_vp_min,slice_vp_max)

fig, ax = plt.subplots(1,figsize=(8,6))
sc = ax.scatter(gxyc[:,0],gxyc[:,1],s=1,c=lores_surf.flatten(),cmap=plt.cm.jet,norm=slice_norm,zorder=0)
ax.fill(dill_bbox.getCLoop()[:,0],dill_bbox.getCLoop()[:,1],c='white',zorder=1)
ax.scatter(slxy[:,0],slxy[:,1],s=1,c=slice_surf.flatten(),cmap=plt.cm.jet,norm=slice_norm,zorder=2)
ax.set_title('Full NAM Model Surface (z=0)')
fig.colorbar(sc)
plt.show()

### Write Specfem Mesh

In [ ]:
from gnam.specutils.gmutil import gmutil

dpath = '../../../data_notebooks/SPECFEM3D/project/'
fname = 'MESH-default-200m'
ofqfname = dpath + fname
print('Writing Mesh to:',ofqfname)

gmu = gmutil()
gmu.writeSpecfem3DMesh(ofqfname, slgm3d)

### Write VTK file

In [ ]:
from gnam.vtkutils.write import write_vtk_gridded_model_3d

dpath = '../../../data_notebooks/SPECFEM3D/project/'
fname = 'fixed_rect_rot_z200m_sig250m'
vtkfqpname = dpath + fname
print('Writing Vtk to:',vtkfqpname)

rs_props = slgm3d.getNPArray()
print('rs_props.shape:',rs_props.shape)
rxdata = np.zeros((3))
rydata = np.zeros((3))
rzdata = np.zeros((3))
rxdata[1] = slgm3d.get_deltas()[0]
rydata[1] = slgm3d.get_deltas()[1]
rzdata[1] = slgm3d.get_deltas()[2]
rxdata[2] = rs_props.shape[1]
rydata[2] = rs_props.shape[2]
rzdata[2] = rs_props.shape[3]
rzdata[0] = slgm3d.get_gorigin()[2]
print('rxdata:',rxdata)
print('rydata:',rydata)
print('rzdata:',rzdata)
write_vtk_gridded_model_3d(vtkfqpname,rs_props,rxdata,rydata,rzdata)